# Predicting house selling prices in Denmark



## Initial overview of steps:
* Guiding research question(s)
* Scrape real estate agency websites (gathering)
* Load data and organize in tidy format (wrangling)
* Deal with data issues (wrangling)
* Exploratory analysis
* Focussed questions
* Explanatory analysis
* Prediction models

## Questions
* How can we predict home prices?


* Is it possible to predict listing prices based on characteristics of the home?
* If so, what features are most important?
* Which ones doesn't matter at all?

# Notes 
The CRISP-DM Process (Cross Industry Process for Data Mining)
The lessons leading up to the first project are about helping you go through CRISP-DM in practice from start to finish. Even when we get into the weeds of coding, try to take a step back and realize what part of the process you are in, and assure that you remember the question you are trying answer and what a solution to that question looks like.

1. Business Understanding

2. Data Understanding

3. Prepare Data

4. Data Modeling

5. Evaluate the Results

6. Deploy

In [1]:
# Importing libraries
import pandas as pd
import requests
import bs4
import time

Browsing Home, the largest real estate company in Denmark and playing arround with the developer tools, I managed to find HTTP call that seem to return the data of the listings.

In [10]:
# Using the home, the biggest real estate company in Denmark
#url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=10&q=2200%20K%C3%B8benhavn%20N&Energimaerker=null&SearchType=0&_=1571481546474'
url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=10&q=2200&Energimaerker=null&SearchType=0&_=1571481546474'
response = requests.get(url)
# Saving response to a dictionary
featuresDict = response.json()

In [12]:
# Checking our the data
featuresDict

{'redirectUrl': None,
 'inputModel': {'SortType': None,
  'SortOrder': None,
  'CurrentPageNumber': 0,
  'SearchResultsPerPage': 10,
  'q': '2200',
  'EjendomstypeV1': None,
  'EjendomstypeRH': None,
  'EjendomstypeEL': None,
  'EjendomstypeVL': None,
  'EjendomstypeAA': None,
  'EjendomstypePL': None,
  'EjendomstypeFH': None,
  'EjendomstypeLO': None,
  'EjendomstypeHG': None,
  'EjendomstypeFG': None,
  'EjendomstypeNL': None,
  'Forretningnr': None,
  'ProjectNodeId': None,
  'OnlyBrokerHome': None,
  'PriceMin': None,
  'PriceMax': None,
  'EjerudgiftPrMdrMin': None,
  'EjerudgiftPrMdrMax': None,
  'BoligydelsePrMdrMin': None,
  'BoligydelsePrMdrMax': None,
  'BoligstoerrelseMin': None,
  'BoligstoerrelseMax': None,
  'GrundstoerrelseMin': None,
  'GrundstoerrelseMax': None,
  'VaerelserMin': None,
  'VaerelserMax': None,
  'Energimaerker': ['null'],
  'ByggaarMin': None,
  'ByggaarMax': None,
  'EtageMin': None,
  'EtageMax': None,
  'PlanMin': None,
  'PlanMax': None,
  'Aabenth

What we want to extract seem to be withing the searchResult key:

In [13]:
featuresDict['searchResults']

[{'sagsnummer': '1050000139',
  'lng': 12.5457172243703,
  'lat': 55.6924852361034,
  'fokusbolig': False,
  'showNewPrice': False,
  'isNew': True,
  'adresse': 'Bjelkes Allé 6B, st..',
  'postal': 2200,
  'city': 'København N',
  'price': '2.095.000 ',
  'ejendomstypePrimaerNicename': 'Ejerlejlighed',
  'pictures': [{'PicId': 2993530,
    'CaseId': 10397003,
    'CaseNumber': '1050000139',
    'MediaType': 'b',
    'MaxWidth': 3000,
    'MaxHeight': 2000,
    'URL': 'https://home.mindworking.eu/resources/shops/105/cases/1050000139/casemedia/images/7687715b8b7896b4ff855797e16a8061/customsize.jpg?deviceId=jd83hsdf3',
    'Position': 0,
    'Description': 'Stue',
    'GUID': '7687715b-8b78-96b4-ff85-5797e16a8061',
    'refGUID': '00000000-0000-0000-0000-000000000000',
    'IsVertical': False,
    'IsHorizontal': True},
   {'PicId': 2993537,
    'CaseId': 10397003,
    'CaseNumber': '1050000139',
    'MediaType': 'b',
    'MaxWidth': 3000,
    'MaxHeight': 2000,
    'URL': 'https://home.

Great! This is the data we're interested in. However the pictures key contain a list of information, we don't need which would ruin the granularity should we convert it to a pandas Dataframe so let's drop it.

In [14]:
# dropping the pictures key from the list of dictionaries
features = featuresDict['searchResults']
for f in features:
    del f['pictures']
features

[{'sagsnummer': '1050000139',
  'lng': 12.5457172243703,
  'lat': 55.6924852361034,
  'fokusbolig': False,
  'showNewPrice': False,
  'isNew': True,
  'adresse': 'Bjelkes Allé 6B, st..',
  'postal': 2200,
  'city': 'København N',
  'price': '2.095.000 ',
  'ejendomstypePrimaerNicename': 'Ejerlejlighed',
  'floorPlan': {'PicId': 2993542,
   'CaseId': 10397003,
   'CaseNumber': '1050000139',
   'MediaType': 'p',
   'MaxWidth': 3000,
   'MaxHeight': 2000,
   'URL': 'https://home.mindworking.eu/resources/shops/105/cases/1050000139/casemedia/images/2f0b1e7e3e1981c99f5d514ebf3f9869/customsize.jpg?deviceId=jd83hsdf3',
   'Position': 0,
   'Description': 'Plantegning',
   'GUID': '2f0b1e7e-3e19-81c9-9f5d-514ebf3f9869',
   'refGUID': '00000000-0000-0000-0000-000000000000',
   'IsVertical': False,
   'IsHorizontal': True},
  'boligOrGrundAreal': 54,
  'andenmaegler': False,
  'boligurl': 'https://home.dk/boligkatalog/koebenhavn/2200/ejerlejligheder/bjelkes_alle_6b_st_1050000139.aspx',
  'billede

The data seem ready to be loaded to a pandas dataframe.

In [15]:
df = pd.DataFrame(features)
df.head()

,aabenthusNicename,aabenthusShowRegistration,adresse,andenmaegler,billedeUrl,boligKanLejes,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,...,lejePerMaaned,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer,showNewPrice,solgtBolig
0,27.10 kl. 12.00-12.30,False,"Bjelkes Allé 6B, st..",False,https://home.mindworking.eu/resources/shops/10...,0,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.545717,2019-10-27T12:30,2019-10-27T12:00,,2200,2.095.000,1050000139,False,False
1,27.10 kl. 14.30-14.50,False,"Poppelgade 4, 1. th.",False,https://home.mindworking.eu/resources/shops/10...,0,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,...,,12.559357,2019-10-27T14:50,2019-10-27T14:30,Beliggende i baghuset,2200,1.799.000,1050000162,False,False
2,27.10 kl. 13.30-13.50,False,"Husumgade 20, 2. th.",False,https://home.mindworking.eu/resources/shops/10...,0,53,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.545400,2019-10-27T13:50,2019-10-27T13:30,Et super godt køb!,2200,2.399.000,1050000164,False,False
3,27.10 kl. 13.30-13.50,False,"Egegade 2, 1. th.",False,https://home.mindworking.eu/resources/shops/10...,0,78,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.559457,2019-10-27T13:50,2019-10-27T13:30,Med altan og stort badeværelse,2200,3.999.000,1050000167,False,False
4,27.10 kl. 11.00-11.20,False,"Fredensborggade 2, 1. th.",False,https://home.mindworking.eu/resources/shops/10...,0,56,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.539880,2019-10-27T11:20,2019-10-27T11:00,Super beliggenhed på Nørrebro,2200,2.199.000,1050000137,False,False


Let's remove columns that are not of interest.

In [16]:
df.drop(inplace = True, columns=[
    'billedeUrl','lejePerMaaned','showNewPrice',
    'aabenthusNicename','floorPlan','erSolgtOgLejebolig',
    'boligKanLejes','aabenthusShowRegistration', 
    'solgtBolig','isLejebolig','fokusbolig'
])

In [17]:
df.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"Bjelkes Allé 6B, st..",False,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.692485,12.545717,2019-10-27T12:30,2019-10-27T12:00,,2200,2.095.000,1050000139
1,"Poppelgade 4, 1. th.",False,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,True,55.692049,12.559357,2019-10-27T14:50,2019-10-27T14:30,Beliggende i baghuset,2200,1.799.000,1050000162
2,"Husumgade 20, 2. th.",False,53,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.693495,12.545400,2019-10-27T13:50,2019-10-27T13:30,Et super godt køb!,2200,2.399.000,1050000164
3,"Egegade 2, 1. th.",False,78,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.690345,12.559457,2019-10-27T13:50,2019-10-27T13:30,Med altan og stort badeværelse,2200,3.999.000,1050000167
4,"Fredensborggade 2, 1. th.",False,56,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.698624,12.539880,2019-10-27T11:20,2019-10-27T11:00,Super beliggenhed på Nørrebro,2200,2.199.000,1050000137


The 'boligurl' is the URL to the site of each piece of real estate for sale, so let's use that to get more features!

In [18]:
response = requests.get(df['boligurl'][0])
html = response.text

In [22]:
html

'\r\n<!DOCTYPE html>\r\n<html lang="da" class="no-js" ng-app="home" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#">\r\n<head>\r\n    <script id="CookieConsent" src="https://policy.cookieinformation.com/uc.js" data-culture="DA" async></script>\r\n    \r\n<script>(function(H){H.className=H.className.replace(/\\bno-js\\b/,\'js\')})(document.documentElement)</script>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\r\n<meta id="viewport" name="viewport" content="width=device-width, initial-scale=1, maximum-scale=2">\r\n<meta name="format-detection" content="telephone=no">\r\n<title>Ejerlejlighed - 2200 København N - Bjelkes Allé 6B, st..</title>\r\n<meta name="title" content="Ejerlejlighed - 2200 København N - Bjelkes Allé 6B, st..">\r\n<meta name="keywords" content="" />\r\n<meta name="description" content="Ejerlejlighed til salg, København N - Førstehåndsindtrykket er rigtig godt, når I træder indenfor i entréen, for allere

The stuff we want is in the info-property and info-value class.

In [39]:
soup = bs4.BeautifulSoup(html, "html.parser")
additionalFeatures = soup.find_all('span', {"class": ["info-property","info-value"]})

[<span class="info-property">Kontantpris</span>,
 <span class="info-value"><b>3.650.000  kr.</b></span>,
 <span class="info-property">Ejerudgift pr. md.</span>,
 <span class="info-value"><b>2.356  kr.</b></span>,
 <span class="info-property">Kvm. pris <i class="tipso" title="Kvm-prisen er baseret på et vægtet areal,  som er mere præcist, fordi der også tages højde for kælderarealer, loftsarealer, udhuse etc. - og ikke kun boligareal. ">?</i></span>,
 <span class="info-value"><b>40.109  kr.</b></span>,
 <span class="info-property">Udbetaling</span>,
 <span class="info-value"><b>185.000  kr.</b></span>,
 <span class="info-property">
                         Brutto/Netto
                         <i class="tipso" title="I brutto- og nettoydelsen indgår standardfinansiering. Da der er tale om en standardfinansiering, vil den i visse tilfælde ikke kunne opnås, hvorfor brutto- og nettoydelsen i så fald kan afvige.">?</i>
 <br>
                         ekskl. ejerudgift
                     </

They come in pairs and we need them divivded into key-value pairs.

In [64]:
# Loop through each span in the list
#import json
count = 0
keys = []
values = []
for feat in additionalFeatures:
    if count % 2: # Odd number is a value
        values.append(feat.text.strip())
        #values.append(re.findall('<b>.+</b>',str(feat))[0][3:-4])
    else: # Even number is a key
        keys.append(feat.text.strip())
        #keys.append(re.findall('>.+<',str(feat))[0][1:-1])
    count +=1 
dictionary = dict(zip(keys, values))
dictionary

{'Kontantpris': '3.650.000  kr.',
 'Ejerudgift pr. md.': '2.356  kr.',
 'Kvm. pris ?': '40.109  kr.',
 'Udbetaling': '185.000  kr.',
 'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift': '14.114  / 12.357  kr.',
 'Prisudvikling': '0%',
 'Boligareal': '91  m2',
 'Grundareal': '570  m2',
 'Antal toiletter': '1',
 'Antal rum': '3',
 'Byggeår': '1906',
 'Energimærke': 'D',
 'Sagsnr.': '1050000133',
 'Afstand til off. transport': '200  m',
 'Afstand til skole': '500  m',
 'Afstand til indkøb': '300  m',
 'Ydermur': 'Mursten',
 'Gulve': 'Plankegulve',
 'Vinduer': 'Termo',
 'El': 'HPFI-relæ',
 'Forurening': 'Jf. udskrift fra RegionH',
 'Overtagelse': 'Efter aftale',
 'Antenne': 'Kabel-tv',
 'Vaskeri': 'Ja',
 'Udlejning tilladt': 'Ja, jf. vedtægterne',
 'Tilbehør': 'Indesit opvaskemaskineGram køleskabVoss ovn',
 'Ejendomsværdi i kr.': '1.600.000',
 'Heraf grundværdi i kr.': '112.200',
 'Vurderingsår': '2018'}

This should be repeated for each line in the dataframe and to be appended as columns. Let's create a function for this.

In [25]:
def get_additional_features(df):
    """Function for getting additional features from each of the listings. Input is the dataframe."""
    additionalFeaturesList = []
    counter = 0
    loops = df.shape[0]
    # Loop through all rows
    for i in df['boligurl']:
        try:
            response = requests.get(i)
            html = response.text
            soup = bs4.BeautifulSoup(html, "html.parser")
            additionalFeatures = soup.find_all('span', {"class": ["info-property","info-value"]})

            # Loop through each span in the list
            count = 0
            keys = []
            values = []
            for feat in additionalFeatures:
                if count % 2: # Odd number is a value
                    values.append(feat.text.strip())
                else: # Even number is a key
                    keys.append(feat.text.strip())
                count +=1 
        except:
            keys.append('Connection timed out')
            values.append('True')
            
        additionalFeaturesList.append(dict(zip(keys, values)))
        time.sleep(2)
        counter += 1
        print('Progress {}'.format((float(counter)/float(loops))*100.))
    df2 = df.join(pd.DataFrame(additionalFeaturesList))
    return df2

In [86]:
df2 = get_additional_features(df)
df2.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,...,Sagsnr.,Teknisk pris ?,Tilbehør,Udbetaling,Udlejning,Udlejning tilladt,Vaskeri,Vinduer,Vurderingsår,Ydermur
0,"Bjelkes Allé 6B, st..",False,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.692485,12.545717,2019-10-27T12:30,...,1050000139,NaN,Gorenje komfurAEG køle/fryseskabBosch emhætteh...,105.000 kr.,NaN,Tilladt,Ja,NaN,2018,Mursten
1,"Poppelgade 4, 1. th.",False,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,True,55.692049,12.559357,2019-10-27T14:50,...,1050000162,3.879.803 kr.,Bosch køle/fryseskabAEG vaskemaskine,NaN,NaN,"Tilladt i kortere periode, jf. vedtægternes § ...",Ja,Termo,2018,Mursten
2,"Husumgade 20, 2. th.",False,53,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.693495,12.545400,2019-10-27T13:50,...,1050000164,NaN,Afventer oplysninger fra sælger,120.000 kr.,Tilladt,NaN,Fællesvaskeri,Termo,2018,Pudset mursten
3,"Egegade 2, 1. th.",False,78,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.690345,12.559457,2019-10-27T13:50,...,1050000167,NaN,Gram køle/fryseskabSiemens komfurElectrolux va...,200.000 kr.,NaN,Med tilladelse fra ejerforeningens bestyrelse,Nej,Termo,2018,Mursten
4,"Fredensborggade 2, 1. th.",False,56,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,True,55.698624,12.539880,2019-10-27T11:20,...,1050000137,NaN,Køleskab fra Blomberg (A+)Komfur fra SMEG,110.000 kr.,Tilladt,NaN,Fællesvaskeri,Termo,2018,Mursten


In [88]:
df2.columns

Index(['adresse', 'andenmaegler', 'boligOrGrundAreal', 'boligurl', 'city',
       'ejendomstypePrimaerNicename', 'isNew', 'lat', 'lng',
       'openHouseEndDate', 'openHouseStartDate', 'overskrift2', 'postal',
       'price', 'sagsnummer', 'Afstand til indkøb',
       'Afstand til off. transport', 'Afstand til skole', 'Altan',
       'Antal plan', 'Antal rum', 'Antal toiletter', 'Antenne', 'Boligareal',
       'Boligydelse pr. måned',
       'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift',
       'Byggeår', 'Ejendomsværdi i kr.', 'Ejerudgift pr. md.', 'El',
       'Energimærke', 'Etage', 'Fibernet', 'Forurening', 'Grundareal', 'Gulve',
       'Heraf grundværdi i kr.', 'Husdyr', 'Husdyr tilladt', 'Kontantpris',
       'Kvm. pris ?', 'Købspris', 'Overtagelse', 'Prisudvikling', 'Pulterrum',
       'Sagsnr.', 'Teknisk pris ?', 'Tilbehør', 'Udbetaling', 'Udlejning',
       'Udlejning tilladt', 'Vaskeri', 'Vinduer', 'Vurderingsår', 'Ydermur'],
     

Alright, we can now do this entire process for multiple zip codes and more than 10 returns.

Note: Through trial and error I found the maximum number of returns to be 200 and in order to get all the data, we can use the URL to add search criteria to split our results into smaller bins.

In [141]:
# Zip codes in Denmark
#zipCode = [2200, 9210]
zipCode = [1301,2000,2100,2200,2300,2400,2450,2500,2600,2605,2610,2625,2630,
           2635,2640,2650,2660,2665,2670,2670,2680,2690,2700,2720,2730,2740,
           2750,2760,2765,2770,2791,2800,2820,2830,2840,2850,2860,2880,2900,
           2920,2930,2942,2950,2960,2970,2980,2990,3000,3050,3060,3070,3080,
           3100,3120,3140,3150,3200,3210,3220,3230,3250,3300,3310,3320,3330,
           3360,3370,3390,3400,3460,3480,3490,3500,3520,3540,3550,3600,3630,
           3650,3660,3670,3700,3720,3730,3740,3751,3760,3770,3782,3790,4000,
           4040,4050,4060,4070,4100,4130,4140,4160,4171,4173,4174,4180,4190,
           4200,4220,4230,4241,4242,4243,4250,4261,4262,4270,4281,4291,4293,
           4295,4296,4300,4320,4330,4340,4350,4360,4370,4390,4400,4420,4440,
           4450,4460,4470,4480,4490,4500,4520,4532,4534,4540,4550,4560,4571,
           4572,4573,4581,4583,4591,4592,4593,4600,4621,4622,4623,4632,4640,
           4652,4653,4654,4660,4671,4672,4673,4681,4682,4683,4684,4690,4700,
           4720,4733,4735,4736,4750,4760,4771,4772,4773,4780,4791,4792,4793,
           4800,4840,4850,4862,4863,4871,4872,4873,4874,4880,4891,4892,4894,
           4895,4900,4912,4913,4920,4930,4941,4943,4944,4951,4952,4953,4960,
           4970,4983,4990,5000,5200,5210,5220,5230,5240,5250,5260,5270,5290,
           5300,5330,5350,5370,5380,5390,5400,5450,5462,5463,5464,5466,5471,
           5474,5485,5491,5492,5500,5540,5550,5560,5580,5591,5592,5600,5610,
           5620,5631,5642,5672,5683,5690,5700,5750,5762,5771,5772,5792,5800,
           5853,5854,5856,5863,5871,5874,5881,5882,5883,5884,5892,5900,5932,
           5935,5953,5960,5970,5985,6000,6040,6051,6052,6064,6070,6091,6092,
           6093,6094,6100,6200,6230,6240,6261,6270,6280,6300,6310,6320,6330,
           6340,6360,6372,6392,6400,6430,6440,6470,6500,6510,6520,6535,6541,
           6560,6580,6600,6621,6622,6623,6630,6640,6650,6660,6670,6682,6683,
           6690,6700,6701,6705,6710,6715,6720,6731,6740,6752,6760,6771,6780,
           6792,6800,6818,6823,6830,6840,6851,6852,6853,6854,6855,6857,6862,
           6870,6880,6893,6900,6920,6933,6940,6950,6960,6971,6973,6980,6990,
           7000,7080,7100,7120,7130,7140,7150,7160,7171,7173,7182,7183,7184,
           7190,7200,7250,7260,7270,7280,7300,7321,7323,7330,7361,7362,7400,
           7430,7441,7442,7451,7470,7480,7490,7500,7540,7550,7560,7570,7600,
           7620,7650,7660,7673,7680,7700,7730,7741,7742,7752,7755,7760,7770,
           7790,7800,7830,7840,7850,7860,7870,7884,7900,7950,7960,7970,7980,
           7990,8000,8200,8210,8220,8230,8240,8250,8260,8270,8300,8305,8310,
           8320,8330,8340,8350,8355,8361,8362,8370,8380,8381,8382,8400,8410,
           8420,8444,8450,8462,8464,8471,8472,8500,8520,8530,8541,8543,8544,
           8550,8560,8570,8581,8585,8586,8592,8600,8620,8632,8641,8643,8653,
           8654,8660,8670,8680,8700,8721,8722,8723,8732,8740,8751,8752,8762,
           8763,8765,8766,8781,8783,8800,8830,8831,8832,8840,8850,8860,8870,
           8881,8882,8883,8900,8950,8961,8963,8970,8981,8983,8990,9000,9200,
           9210,9220,9230,9240,9260,9270,9280,9293,9300,9310,9320,9330,9340,
           9352,9362,9370,9380,9381,9382,9400,9430,9440,9460,9480,9490,9492,
           9493,9500,9510,9520,9530,9541,9550,9560,9574,9575,9600,9610,9620,
           9631,9632,9640,9670,9681,9690,9700,9740,9750,9760,9800,9830,9850,
           9870,9881,9900,9940,9970,9981,9982,9990
          ]


In [142]:

featureList = []
# Loop through zip codes
for code in zipCode:
    # If the zipcode is in one of the larger cities, split the search into chunks based on size
    if code in [1301, 2000, 2100, 2200, 2300, 2400, 2450, 2500,
                5000, 5200, 5210, 5220, 5230, 5240, 5250, 5260,
                5270, 8000, 8200, 8210, 8220, 8230, 8240, 9000,
                9200, 9210, 9220
               ]:
        # Setting size interval to bin responses into smaller chunks
        minSize = 11
        maxSize = 20
        # Loop through sizes
        for i in range(28):
            url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=200&BoligstoerrelseMin=' + str(minSize) + '&BoligstoerrelseMax=' + str(maxSize) + '&q=' + str(code) + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474'

            response = requests.get(url)
            # Saving response to a dictionary
            featuresDict = response.json()
            # dropping the pictures key from the list of dictionaries
            features = featuresDict['searchResults']
            for f in features:
                del f['pictures']
            featureList.extend(features)
            # Pausing to not be a dick towards the server
            time.sleep(1)
            
            # Count up sizes
            minSize += 10
            maxSize += 10
        
        # Run one additional time with out the maximum boundry
        url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=200&BoligstoerrelseMin=' + str(minSize) + '&q=' + str(code) + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474'

        response = requests.get(url)
        # Saving response to a dictionary
        featuresDict = response.json()
        # dropping the pictures key from the list of dictionaries
        features = featuresDict['searchResults']
        for f in features:
            del f['pictures']
        featureList.extend(features)
    # If the zipcode not in a larger city
    else:
        url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=200&q=' + str(code) + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474'

        response = requests.get(url)
        # Saving response to a dictionary
        featuresDict = response.json()
        # dropping the pictures key from the list of dictionaries
        features = featuresDict['searchResults']
        for f in features:
            del f['pictures']
        featureList.extend(features)

len(featureList)

51207

In [143]:
df_new = pd.DataFrame(featureList)
df_new.drop(inplace = True, columns=[
    'billedeUrl','lejePerMaaned','showNewPrice',
    'aabenthusNicename','floorPlan','erSolgtOgLejebolig',
    'boligKanLejes','aabenthusShowRegistration', 
    'solgtBolig','isLejebolig','fokusbolig'
])
df_new.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,,,,2000,2.145.000,1740000062
2,"Holger Danskes Vej 12, 3 tv",True,46.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.686592,12.538423,,,None,2000,2.150.000,20002433_10007
3,"Ane Katrines Vej 16, 2 4",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,,,None,2000,2.099.000,303562_100910_10009
4,"Ane Katrines Vej 16, St. 1",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,,,None,2000,2.295.000,100088_103402_10009


In [144]:
df_new.to_csv('baseData.csv')

In [3]:
df_new = pd.read_csv('baseData.csv',index_col = 0 )
df_new.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,NaN,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,NaN,NaN,NaN,2000,2.145.000,1740000062
2,"Holger Danskes Vej 12, 3 tv",True,46.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.686592,12.538423,NaN,NaN,NaN,2000,2.150.000,20002433_10007
3,"Ane Katrines Vej 16, 2 4",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,NaN,NaN,NaN,2000,2.099.000,303562_100910_10009
4,"Ane Katrines Vej 16, St. 1",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,NaN,NaN,NaN,2000,2.295.000,100088_103402_10009


Inspecting the data it's clear that we have some appartments from other real estate agencies, but how many?

In [17]:
df_new.query('andenmaegler == True')['adresse'].count()

41777

Turns out it's actually the majority. It's not very likely the previously created function for getting additional features would work for the remaining URLs.

For now we can get the additional features for the Home listings.

In [18]:
df_new.query('andenmaegler == False')['adresse'].count()

9430

In [19]:
df_home = df_new.query('andenmaegler == False')
df_home.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,NaN,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,NaN,NaN,NaN,2000,2.145.000,1740000062
11,"Lyøvej 5, st.. tv.",False,60.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,NaN,NaN,HØJ STUELEJLIGHED MED STOR ALTAN - LAV EJERUDG...,2000,2.875.000,1300000128
12,"H. Schneekloths Vej 13, 5. th.",False,56.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,NaN,NaN,Lys og indflytningsklar stand – perfekt delele...,2000,2.750.000,130D01015
30,"Howitzvej 61, 3. th.",False,67.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2019-11-03T15:20,2019-11-03T15:00,Nyere bad & køkken - i kort afstand til Freder...,2000,3.195.000,1300000176


In [26]:
df_home_add = get_additional_features(df_home)

Progress 0.010604453870625662
Progress 0.021208907741251323
Progress 0.03181336161187699
Progress 0.04241781548250265
Progress 0.05302226935312832
Progress 0.06362672322375398
Progress 0.07423117709437964
Progress 0.0848356309650053
Progress 0.09544008483563096
Progress 0.10604453870625664
Progress 0.11664899257688228
Progress 0.12725344644750797
Progress 0.1378579003181336
Progress 0.14846235418875928
Progress 0.15906680805938495
Progress 0.1696712619300106
Progress 0.18027571580063625
Progress 0.19088016967126192
Progress 0.20148462354188762
Progress 0.2120890774125133
Progress 0.2226935312831389
Progress 0.23329798515376457
Progress 0.24390243902439024
Progress 0.25450689289501593
Progress 0.2651113467656416
Progress 0.2757158006362672
Progress 0.28632025450689286
Progress 0.29692470837751855
Progress 0.3075291622481442
Progress 0.3181336161187699
Progress 0.3287380699893956
Progress 0.3393425238600212
Progress 0.34994697773064687
Progress 0.3605514316012725
Progress 0.3711558854718

Progress 3.138918345705196
Progress 3.149522799575822
Progress 3.160127253446447
Progress 3.1707317073170733
Progress 3.1813361611876987
Progress 3.1919406150583245
Progress 3.2025450689289503
Progress 3.2131495227995757
Progress 3.223753976670202
Progress 3.234358430540827
Progress 3.244962884411453
Progress 3.255567338282078
Progress 3.2661717921527043
Progress 3.2767762460233296
Progress 3.2873806998939554
Progress 3.2979851537645812
Progress 3.3085896076352066
Progress 3.319194061505833
Progress 3.3297985153764578
Progress 3.340402969247084
Progress 3.3510074231177094
Progress 3.361611876988335
Progress 3.3722163308589606
Progress 3.3828207847295864
Progress 3.3934252386002126
Progress 3.4040296924708375
Progress 3.414634146341464
Progress 3.425238600212089
Progress 3.435843054082715
Progress 3.4464475079533403
Progress 3.457051961823966
Progress 3.4676564156945915
Progress 3.4782608695652173
Progress 3.4888653234358427
Progress 3.4994697773064685
Progress 3.5100742311770947
Progre

Progress 6.341463414634147
Progress 6.352067868504772
Progress 6.362672322375397
Progress 6.373276776246023
Progress 6.383881230116649
Progress 6.394485683987275
Progress 6.405090137857901
Progress 6.415694591728525
Progress 6.426299045599151
Progress 6.436903499469778
Progress 6.447507953340404
Progress 6.458112407211028
Progress 6.468716861081654
Progress 6.47932131495228
Progress 6.489925768822906
Progress 6.5005302226935315
Progress 6.511134676564156
Progress 6.521739130434782
Progress 6.5323435843054085
Progress 6.542948038176035
Progress 6.553552492046659
Progress 6.564156945917285
Progress 6.574761399787911
Progress 6.585365853658537
Progress 6.5959703075291625
Progress 6.606574761399788
Progress 6.617179215270413
Progress 6.6277836691410394
Progress 6.638388123011666
Progress 6.64899257688229
Progress 6.6595970307529155
Progress 6.670201484623542
Progress 6.680805938494168
Progress 6.691410392364794
Progress 6.702014846235419
Progress 6.712619300106044
Progress 6.72322375397667

Progress 9.575821845174975
Progress 9.586426299045598
Progress 9.597030752916226
Progress 9.607635206786851
Progress 9.618239660657476
Progress 9.628844114528102
Progress 9.639448568398727
Progress 9.650053022269352
Progress 9.66065747613998
Progress 9.671261930010605
Progress 9.68186638388123
Progress 9.692470837751856
Progress 9.703075291622481
Progress 9.713679745493108
Progress 9.724284199363732
Progress 9.734888653234359
Progress 9.745493107104984
Progress 9.75609756097561
Progress 9.766702014846237
Progress 9.77730646871686
Progress 9.787910922587487
Progress 9.798515376458113
Progress 9.809119830328738
Progress 9.819724284199363
Progress 9.830328738069989
Progress 9.840933191940614
Progress 9.851537645811241
Progress 9.862142099681867
Progress 9.872746553552492
Progress 9.883351007423117
Progress 9.893955461293743
Progress 9.90455991516437
Progress 9.915164369034994
Progress 9.92576882290562
Progress 9.936373276776246
Progress 9.946977730646871
Progress 9.957582184517497
Progres

Progress 12.725344644750795
Progress 12.735949098621422
Progress 12.746553552492045
Progress 12.757158006362673
Progress 12.767762460233298
Progress 12.778366914103923
Progress 12.78897136797455
Progress 12.799575821845174
Progress 12.810180275715801
Progress 12.820784729586427
Progress 12.83138918345705
Progress 12.841993637327679
Progress 12.852598091198303
Progress 12.86320254506893
Progress 12.873806998939555
Progress 12.884411452810179
Progress 12.895015906680808
Progress 12.905620360551431
Progress 12.916224814422057
Progress 12.926829268292684
Progress 12.937433722163307
Progress 12.948038176033934
Progress 12.95864262990456
Progress 12.969247083775185
Progress 12.979851537645812
Progress 12.990455991516436
Progress 13.001060445387063
Progress 13.011664899257688
Progress 13.022269353128312
Progress 13.032873806998941
Progress 13.043478260869565
Progress 13.054082714740192
Progress 13.064687168610817
Progress 13.07529162248144
Progress 13.08589607635207
Progress 13.09650053022269

...

Progress 86.28844114528101
Progress 86.29904559915164
Progress 86.30965005302227
Progress 86.3202545068929
Progress 86.33085896076352
Progress 86.34146341463415
Progress 86.35206786850476
Progress 86.3626723223754
Progress 86.37327677624602
Progress 86.38388123011664
Progress 86.39448568398727
Progress 86.4050901378579
Progress 86.41569459172854
Progress 86.42629904559915
Progress 86.43690349946978
Progress 86.4475079533404
Progress 86.45811240721103
Progress 86.46871686108165
Progress 86.47932131495229
Progress 86.4899257688229
Progress 86.50053022269353
Progress 86.51113467656415
Progress 86.52173913043478
Progress 86.53234358430541
Progress 86.54294803817604
Progress 86.55355249204666
Progress 86.56415694591728
Progress 86.57476139978792
Progress 86.58536585365853
Progress 86.59597030752916
Progress 86.60657476139978
Progress 86.61717921527041
Progress 86.62778366914104
Progress 86.63838812301167
Progress 86.64899257688229
Progress 86.65959703075292
Progress 86.67020148462355
Progre

Progress 89.53340402969248
Progress 89.5440084835631
Progress 89.55461293743372
Progress 89.56521739130436
Progress 89.57582184517497
Progress 89.5864262990456
Progress 89.59703075291623
Progress 89.60763520678685
Progress 89.61823966065748
Progress 89.6288441145281
Progress 89.63944856839873
Progress 89.65005302226935
Progress 89.66065747613999
Progress 89.6712619300106
Progress 89.68186638388123
Progress 89.69247083775186
Progress 89.70307529162248
Progress 89.7136797454931
Progress 89.72428419936374
Progress 89.73488865323435
Progress 89.74549310710498
Progress 89.75609756097562
Progress 89.76670201484623
Progress 89.77730646871687
Progress 89.78791092258749
Progress 89.79851537645811
Progress 89.80911983032874
Progress 89.81972428419937
Progress 89.83032873806998
Progress 89.84093319194062
Progress 89.85153764581125
Progress 89.86214209968186
Progress 89.8727465535525
Progress 89.88335100742312
Progress 89.89395546129374
Progress 89.90455991516437
Progress 89.915164369035
Progress 

Progress 92.77836691410393
Progress 92.78897136797455
Progress 92.79957582184517
Progress 92.81018027571581
Progress 92.82078472958642
Progress 92.83138918345705
Progress 92.84199363732768
Progress 92.8525980911983
Progress 92.86320254506893
Progress 92.87380699893956
Progress 92.88441145281018
Progress 92.89501590668081
Progress 92.90562036055144
Progress 92.91622481442205
Progress 92.92682926829269
Progress 92.9374337221633
Progress 92.94803817603393
Progress 92.95864262990456
Progress 92.96924708377519
Progress 92.97985153764581
Progress 92.99045599151644
Progress 93.00106044538707
Progress 93.01166489925768
Progress 93.02226935312832
Progress 93.03287380699894
Progress 93.04347826086956
Progress 93.05408271474019
Progress 93.06468716861082
Progress 93.07529162248144
Progress 93.08589607635207
Progress 93.0965005302227
Progress 93.10710498409331
Progress 93.11770943796395
Progress 93.12831389183457
Progress 93.1389183457052
Progress 93.14952279957582
Progress 93.16012725344645
Progr

Progress 96.02332979851538
Progress 96.033934252386
Progress 96.04453870625663
Progress 96.05514316012726
Progress 96.06574761399787
Progress 96.07635206786851
Progress 96.08695652173913
Progress 96.09756097560975
Progress 96.10816542948038
Progress 96.118769883351
Progress 96.12937433722163
Progress 96.13997879109226
Progress 96.15058324496289
Progress 96.1611876988335
Progress 96.17179215270414
Progress 96.18239660657476
Progress 96.19300106044538
Progress 96.20360551431601
Progress 96.21420996818664
Progress 96.22481442205726
Progress 96.23541887592789
Progress 96.24602332979852
Progress 96.25662778366915
Progress 96.26723223753977
Progress 96.27783669141039
Progress 96.28844114528103
Progress 96.29904559915164
Progress 96.30965005302227
Progress 96.3202545068929
Progress 96.33085896076352
Progress 96.34146341463415
Progress 96.35206786850478
Progress 96.3626723223754
Progress 96.37327677624602
Progress 96.38388123011666
Progress 96.39448568398727
Progress 96.4050901378579
Progress 

Progress 99.26829268292683
Progress 99.27889713679745
Progress 99.28950159066808
Progress 99.30010604453871
Progress 99.31071049840932
Progress 99.32131495227996
Progress 99.33191940615058
Progress 99.3425238600212
Progress 99.35312831389183
Progress 99.36373276776246
Progress 99.37433722163308
Progress 99.38494167550371
Progress 99.39554612937434
Progress 99.40615058324497
Progress 99.41675503711559
Progress 99.4273594909862
Progress 99.43796394485685
Progress 99.44856839872746
Progress 99.45917285259809
Progress 99.46977730646871
Progress 99.48038176033934
Progress 99.49098621420997
Progress 99.5015906680806
Progress 99.51219512195122
Progress 99.52279957582184
Progress 99.53340402969248
Progress 99.54400848356309
Progress 99.55461293743372
Progress 99.56521739130434
Progress 99.57582184517497
Progress 99.5864262990456
Progress 99.59703075291623
Progress 99.60763520678685
Progress 99.61823966065748
Progress 99.6288441145281
Progress 99.63944856839872
Progress 99.65005302226936
Progre

In [27]:
df_home_add.to_csv('home_data.csv')

In [16]:
df_new.query('andenmaegler == True')['boligurl'].values

array(['https://www.boligsiden.dk/viderestillingekstern/bc48e4ab0904446388322b86cc3dee05?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10007_200',
       'https://www.boligsiden.dk/viderestillingekstern/c31026c2150b45dd96717daefe5e0b55?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10009_100910',
       'https://www.boligsiden.dk/viderestillingekstern/5837908bee574eccbc87ec74143cb9f4?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10009_103402',
       ...,
       'https://www.boligsiden.dk/viderestillingekstern/bd4d042ee8b641dd80e747bfd9886be4?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10007_999',
       'https://www.boligsiden.dk/viderestillingekstern/c1e68e6264cb4d2fbc5fac9701e630a8?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10007_999',
       'https://www.boligsiden.dk/viderestillingekstern/438dde6c110a43b3ae5157d96e8a86f0?extid=TxKr9IXWfkOCgtEMyQVciQ&aid=10010_1561'],
      dtype=object)

In [ ]:
df_new2 = get_additional_features(df_new)

0.0019528580076942606
0.0039057160153885212
0.005858574023082781
0.0078114320307770425
0.009764290038471303
0.011717148046165563
0.013670006053859825
0.015622864061554085
0.017575722069248345
0.019528580076942605
0.021481438084636865
0.023434296092331126
0.02538715410002539
0.02734001210771965
0.029292870115413906
0.03124572812310817
0.03319858613080243
0.03515144413849669
0.03710430214619095
0.03905716015388521
0.041010018161579474
0.04296287616927373
0.044915734176967995
0.04686859218466225
0.048821450192356515
0.05077430820005078
0.052727166207745035
0.0546800242154393
0.05663288222313356
0.05858574023082781
0.060538598238522076
0.06249145624621634
0.0644443142539106
0.06639717226160485
0.06835003026929912
0.07030288827699338
0.07225574628468763
0.0742086042923819
0.07616146230007616
0.07811432030777042
0.08006717831546468
0.08202003632315895
0.08397289433085321
0.08592575233854746
0.08787861034624173
0.08983146835393599
0.09178432636163025
0.0937371843693245
0.09569004237701877
0.0

0.8299646532700606
0.8319175112777549
0.8338703692854492
0.8358232272931434
0.8377760853008377
0.839728943308532
0.8416818013162263
0.8436346593239206
0.8455875173316147
0.847540375339309
0.8494932333470033
0.8514460913546975
0.8533989493623918
0.8553518073700861
0.8573046653777804
0.8592575233854747
0.8612103813931689
0.8631632394008631
0.8651160974085574
0.8670689554162516
0.8690218134239459
0.8709746714316402
0.8729275294393345
0.8748803874470288
0.876833245454723
0.8787861034624173
0.8807389614701115
0.8826918194778057
0.8846446774855
0.8865975354931943
0.8885503935008886
0.8905032515085829
0.8924561095162771
0.8944089675239714
0.8963618255316657
0.8983146835393598
0.9002675415470541
0.9022203995547484
0.9041732575624427
0.906126115570137
0.9080789735778312
0.9100318315855255
0.9119846895932198
0.913937547600914
0.9158904056086082
0.9178432636163025
0.9197961216239968
0.9217489796316911
0.9237018376393853
0.9256546956470796
0.9276075536547739
0.929560411662468
0.9315132696701622
0.

1.6833636026324525
1.6853164606401467
1.6872693186478411
1.6892221766555353
1.6911750346632295
1.693127892670924
1.695080750678618
1.6970336086863123
1.6989864666940067
1.7009393247017008
1.702892182709395
1.7048450407170894
1.7067978987247836
1.708750756732478
1.7107036147401722
1.7126564727478664
1.7146093307555608
1.716562188763255
1.7185150467709494
1.7204679047786435
1.7224207627863377
1.7243736207940321
1.7263264788017263
1.7282793368094205
1.7302321948171149
1.732185052824809
1.7341379108325032
1.7360907688401976
1.7380436268478918
1.7399964848555862
1.7419493428632804
1.7439022008709746
1.745855058878669
1.7478079168863632
1.7497607748940576
1.7517136329017517
1.753666490909446
1.7556193489171403
1.7575722069248345
1.7595250649325287
1.761477922940223
1.7634307809479173
1.7653836389556115
1.7673364969633059
1.769289354971
1.7712422129786944
1.7731950709863886
1.7751479289940828
1.7771007870017772
1.7790536450094714
1.7810065030171658
1.78295936102486
1.7849122190325541
1.786865

2.5465268420333156
2.54847970004101
2.550432558048704
2.5523854160563983
2.5543382740640928
2.556291132071787
2.558243990079481
2.5601968480871755
2.56214970609487
2.564102564102564
2.5660554221102583
2.5680082801179527
2.5699611381256466
2.571913996133341
2.5738668541410354
2.5758197121487294
2.577772570156424
2.579725428164118
2.5816782861718126
2.5836311441795066
2.585584002187201
2.5875368601948954
2.5894897182025893
2.5914425762102837
2.593395434217978
2.595348292225672
2.5973011502333665
2.599254008241061
2.601206866248755
2.6031597242564493
2.6051125822641437
2.6070654402718376
2.609018298279532
2.6109711562872264
2.6129240142949204
2.614876872302615
2.616829730310309
2.6187825883180036
2.6207354463256975
2.622688304333392
2.6246411623410864
2.6265940203487803
2.6285468783564747
2.630499736364169
2.632452594371863
2.6344054523795575
2.636358310387252
2.638311168394946
2.6402640264026402
2.6422168844103346
2.644169742418029
2.646122600425723
2.6480754584334174
2.650028316441112
2

3.413595797449567
3.4155486554572616
3.417501513464956
3.41945437147265
3.4214072294803444
3.4233600874880388
3.4253129454957327
3.427265803503427
3.4292186615111215
3.4311715195188155
3.43312437752651
3.4350772355342043
3.4370300935418987
3.4389829515495927
3.440935809557287
3.4428886675649815
3.4448415255726754
3.44679438358037
3.4487472415880642
3.450700099595758
3.4526529576034526
3.454605815611147
3.456558673618841
3.4585115316265354
3.4604643896342298
3.4624172476419237
3.464370105649618
3.4663229636573125
3.4682758216650065
3.470228679672701
3.4721815376803953
3.4741343956880897
3.4760872536957836
3.478040111703478
3.4799929697111724
3.4819458277188664
3.483898685726561
3.485851543734255
3.487804401741949
3.4897572597496436
3.491710117757338
3.493662975765032
3.4956158337727263
3.4975686917804207
3.499521549788115
3.501474407795809
3.5034272658035035
3.505380123811198
3.507332981818892
3.5092858398265863
3.5112386978342807
3.5131915558419746
3.515144413849669
3.5170972718573634


...

6.065576971898373
6.067529829906068
6.069482687913761
6.071435545921456
6.07338840392915
6.0753412619368445
6.077294119944539
6.079246977952233
6.081199835959927
6.083152693967621
6.085105551975316
6.08705840998301
6.0890112679907045
6.090964125998399
6.092916984006093
6.094869842013787
6.096822700021481
6.098775558029176
6.10072841603687
6.102681274044564
6.104634132052259
6.106586990059952
6.108539848067647
6.110492706075341
6.1124455640830355
6.11439842209073
6.116351280098424
6.118304138106119
6.120256996113812
6.122209854121507
6.124162712129201
6.1261155701368954
6.12806842814459
6.130021286152284
6.131974144159978
6.133927002167672
6.135879860175367
6.137832718183061
6.139785576190755
6.14173843419845
6.143691292206144
6.145644150213838
6.147597008221532
6.1495498662292265
6.151502724236921
6.153455582244615
6.15540844025231
6.157361298260003
6.159314156267698
6.161267014275392
6.163219872283086
6.165172730290781
6.167125588298475
6.169078446306169
6.171031304313863
6.1729841623

6.954127365399263
6.956080223406956
6.958033081414651
6.959985939422345
6.961938797430039
6.963891655437733
6.965844513445428
6.967797371453122
6.969750229460817
6.97170308746851
6.973655945476205
6.975608803483898
6.977561661491594
6.979514519499287
6.981467377506982
6.983420235514676
6.98537309352237
6.987325951530064
6.989278809537758
6.991231667545453
6.993184525553146
6.9951373835608415
6.997090241568535
6.99904309957623
7.000995957583924
7.002948815591618
7.004901673599312
7.006854531607007
7.0088073896147005
7.010760247622396
7.012713105630089
7.014665963637784
7.016618821645477
7.0185716796531725
7.020524537660866
7.022477395668561
7.024430253676255
7.026383111683949
7.028335969691643
7.030288827699338
7.032241685707032
7.034194543714727
7.03614740172242
7.038100259730115
7.040053117737808
7.042005975745504
7.043958833753197
7.045911691760892
7.047864549768586
7.04981740777628
7.051770265783974
7.053723123791669
7.055675981799363
7.057628839807058
7.059581697814751
7.0615345558

7.84267775890015
7.844630616907845
7.846583474915539
7.848536332923234
7.850489190930928
7.852442048938622
7.854394906946316
7.856347764954011
7.858300622961704
7.8602534809694
7.862206338977093
7.864159196984787
7.866112054992482
7.868064913000175
7.87001777100787
7.871970629015563
7.873923487023259
7.875876345030952
7.877829203038647
7.879782061046341
7.881734919054035
7.883687777061729
7.885640635069424
7.887593493077118
7.889546351084813
7.8914992090925065
7.893452067100201
7.895404925107894
7.89735778311559
7.899310641123283
7.9012634991309785
7.903216357138672
7.905169215146366
7.90712207315406
7.909074931161755
7.911027789169449
7.912980647177144
7.9149335051848375
7.916886363192532
7.918839221200226
7.920792079207921
7.922744937215614
7.9246977952233095
7.926650653231003
7.928603511238698
7.930556369246392
7.932509227254086
7.93446208526178
7.936414943269475
7.938367801277169
7.940320659284864
7.942273517292557
7.944226375300252
7.946179233307945
7.948132091315641
7.95008494932

8.74099244243951
8.742945300447204
8.744898158454898
8.746851016462593
8.748803874470287
8.750756732477981
8.752709590485676
8.75466244849337
8.756615306501065
8.758568164508757
8.760521022516453
8.762473880524146
8.764426738531842
8.766379596539535
8.76833245454723
8.770285312554924
8.772238170562618
8.774191028570312
8.776143886578007
8.778096744585701
8.780049602593396
8.782002460601088
8.783955318608784
8.785908176616477
8.787861034624173
8.789813892631866
8.79176675063956
8.793719608647255
8.795672466654949
8.797625324662643
8.799578182670338
8.801531040678032
8.803483898685727
8.805436756693421
8.807389614701115
8.809342472708808
8.811295330716504
8.813248188724197
8.815201046731893
8.817153904739586
8.81910676274728
8.821059620754975
8.823012478762669
8.824965336770363
8.826918194778058
8.828871052785752
8.830823910793447
8.83277676880114
8.834729626808835
8.836682484816528
8.838635342824224
8.840588200831917
8.842541058839611
8.844493916847306
8.846446774855
8.848399632862694
8

9.641259983986563
9.64321284199426
9.645165700001952
9.647118558009646
9.64907141601734
9.651024274025035
9.65297713203273
9.654929990040424
9.656882848048118
9.658835706055813
9.660788564063507
9.662741422071202
9.664694280078894
9.66664713808659
9.668599996094283
9.67055285410198
9.672505712109672
9.674458570117366
9.67641142812506
9.678364286132755
9.68031714414045
9.682270002148144
9.684222860155838
9.686175718163533
9.688128576171225
9.690081434178921
9.692034292186614
9.69398715019431
9.695940008202003
9.697892866209697
9.699845724217392
9.701798582225086
9.70375144023278
9.705704298240475
9.70765715624817
9.709610014255864
9.711562872263556
9.713515730271252
9.715468588278945
9.717421446286641
9.719374304294334
9.721327162302028
9.723280020309723
9.725232878317417
9.727185736325112
9.729138594332806
9.7310914523405
9.733044310348195
9.73499716835589
9.736950026363584
9.738902884371276
9.740855742378972
9.742808600386665
9.744761458394361
9.746714316402054
9.748667174409748
9.750

In [ ]:
df_new2.to_csv('extendedData.csv')

In [126]:
df_new2 = GetAdditionalFeatures(df_new.head())
df_new2.head()

20.0
40.0
60.0
80.0
100.0


,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,...,Overtagelse,Prisudvikling,Sagsnr.,Tilbehør,Udbetaling,Udlejning tilladt,Vaskeri,Vinduer,Vurderingsår,Ydermur
0,"Dagmarsgade 36, 1 Lejl. 4",True,32,https://www.boligsiden.dk/viderestillingekster...,København N,Ejerlejlighed,True,55.698600,12.546053,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Åboulevard 34D, 5 th",True,37,https://www.boligsiden.dk/viderestillingekster...,København N,Ejerlejlighed,False,55.684981,12.554957,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Dagmarsgade 36, 4",True,32,https://www.boligsiden.dk/viderestillingekster...,København N,Ejerlejlighed,False,55.698600,12.546053,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Søllerødgade 46, 5 tv",True,37,https://www.boligsiden.dk/viderestillingekster...,København N,Ejerlejlighed,False,55.696019,12.543893,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Slejpnersgade 6, 1. 3.",False,44,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,55.701175,12.543234,2019-10-27T16:20,...,Efter aftale,-5%,1050000126,Whirlpool vaskemaskineGorenje gaskomfur - Bauk...,95.000 kr.,"Tilladt, jf. vedtægternes § 13",Vaskemaskine i lejligheden,Termo,2018,Mursten


In [ ]:
df_new2.head()

In [75]:
df2.columns

Index(['adresse', 'andenmaegler', 'boligOrGrundAreal', 'boligurl', 'city',
       'ejendomstypePrimaerNicename', 'isNew', 'lat', 'lng',
       'openHouseEndDate', 'openHouseStartDate', 'overskrift2', 'postal',
       'price', 'sagsnummer', 'Afstand til indkøb',
       'Afstand til off. transport', 'Afstand til skole', 'Altan',
       'Antal plan', 'Antal rum', 'Antal toiletter', 'Antenne', 'Boligareal',
       'Boligydelse pr. måned',
       'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift',
       'Byggeår', 'Ejendomsværdi i kr.', 'Ejerudgift pr. md.', 'El',
       'Energimærke', 'Etage', 'Fibernet', 'Forurening', 'Grundareal', 'Gulve',
       'Heraf grundværdi i kr.', 'Husdyr', 'Husdyr tilladt', 'Kontantpris',
       'Kvm. pris ?', 'Købspris', 'Overtagelse', 'Prisudvikling', 'Pulterrum',
       'Sagsnr.', 'Teknisk pris ?', 'Tilbehør', 'Udbetaling', 'Udlejning',
       'Udlejning tilladt', 'Vaskeri', 'Vinduer', 'Vurderingsår', 'Ydermur'],
     

In [76]:
df2.shape

(10, 55)

In [213]:
feats = dict['searchResults']
url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=200&BoligstoerrelseMin=60&BoligstoerrelseMax=70&q=2200&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474'
response = requests.get(url)    
dict2 = response.json()
feats2 = dict2['searchResults']

In [214]:
len(feats2)

21

In [188]:
for f in feats:
    del f['pictures']

TypeError: list indices must be integers or slices, not str

In [144]:
len(feats)

200

In [165]:
feats[0]['floorPlan']

{'PicId': 2985754,
 'CaseId': 10395211,
 'CaseNumber': '1050000137',
 'MediaType': 'p',
 'MaxWidth': 3000,
 'MaxHeight': 2000,
 'URL': 'https://home.mindworking.eu/resources/shops/105/cases/1050000137/casemedia/images/d648ebc328cc9e08efd3e8f608061497/customsize.jpg?deviceId=jd83hsdf3',
 'Position': 0,
 'Description': 'Plantegning',
 'GUID': 'd648ebc3-28cc-9e08-efd3-e8f608061497',
 'refGUID': '00000000-0000-0000-0000-000000000000',
 'IsVertical': False,
 'IsHorizontal': True}

In [145]:
df = pd.DataFrame(feats)

In [179]:
df.drop(inplace = True, columns=[
    'billedeUrl','lejePerMaaned','showNewPrice',
    'aabenthusNicename','floorPlan','erSolgtOgLejebolig',
    'boligKanLejes','aabenthusShowRegistration', 
    'solgtBolig','isLejebolig','fokusbolig'
])

In [149]:
df.postal.value_counts()

2200    200
Name: postal, dtype: int64

In [180]:
df.nunique()

adresse                        200
andenmaegler                     2
boligOrGrundAreal               80
city                             1
ejendomstypePrimaerNicename      3
isNew                            2
lat                            154
lng                            154
openHouseEndDate                 5
openHouseStartDate               5
overskrift2                     30
postal                           1
price                          114
sagsnummer                     200
dtype: int64

In [70]:
df.nunique()

adresse                        10
andenmaegler                    1
boligOrGrundAreal              10
boligurl                       10
city                            1
ejendomstypePrimaerNicename     2
isNew                           2
lat                            10
lng                            10
openHouseEndDate                8
openHouseStartDate              8
overskrift2                    10
postal                          1
price                          10
sagsnummer                     10
dtype: int64

In [82]:
df.shape

(22, 27)

In [65]:
url = 'https://home.dk/resultatliste/?CurrentPageNumber=0&SearchResultsPerPage=15&q=2200%20K%C3%B8benhavn%20N&Energimaerker=null&SearchType=0'

In [11]:
content_div = soup.find_all('home-tile-info')
content_div

[]

In [21]:
urlreq = 'https://home.dk/umbraco/backoffice/home-api/BoligOrAddress/Boligdata?max=100&searchstring=2200'

In [24]:
# import json library
import json
import urllib.request
# request url
#urlreq = 'https://groceries.asda.com/api/items/search?keyword=yogurt'
# get response
response = urllib.request.urlopen(urlreq)
# load as json
jresponse = json.load(response)
json.loads(line.decode("utf-8","ignore"))
# write to file as pretty print
with open('asdaresp.json', 'w') as outfile:
    json.dump(jresponse, outfile, sort_keys=True, indent=4)
response.read()

b''

In [26]:
req = urllib.request.Request(urlreq)
with urllib.request.urlopen(req) as response:
   the_page = response.read()
print(the_page)

b'{"Successed":true,"Status":"OK","InputModel":{"SearchString":"2200","Max":100},"SuggestItems":[{"suggest":"2200 K\xc3\xb8benhavn N","count":"200","sortorder":40,"IsHeadLine":false}]}'


In [66]:
import requests

url = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=15&q=2200%20K%C3%B8benhavn%20N&Energimaerker=null&SearchType=0&_=1571481546474'
response = requests.get(url)    
dict = response.json()
dict

{'redirectUrl': None,
 'inputModel': {'SortType': None,
  'SortOrder': None,
  'CurrentPageNumber': 0,
  'SearchResultsPerPage': 15,
  'q': '2200 København N',
  'EjendomstypeV1': None,
  'EjendomstypeRH': None,
  'EjendomstypeEL': None,
  'EjendomstypeVL': None,
  'EjendomstypeAA': None,
  'EjendomstypePL': None,
  'EjendomstypeFH': None,
  'EjendomstypeLO': None,
  'EjendomstypeHG': None,
  'EjendomstypeFG': None,
  'EjendomstypeNL': None,
  'Forretningnr': None,
  'ProjectNodeId': None,
  'OnlyBrokerHome': None,
  'PriceMin': None,
  'PriceMax': None,
  'EjerudgiftPrMdrMin': None,
  'EjerudgiftPrMdrMax': None,
  'BoligydelsePrMdrMin': None,
  'BoligydelsePrMdrMax': None,
  'BoligstoerrelseMin': None,
  'BoligstoerrelseMax': None,
  'GrundstoerrelseMin': None,
  'GrundstoerrelseMax': None,
  'VaerelserMin': None,
  'VaerelserMax': None,
  'Energimaerker': ['null'],
  'ByggaarMin': None,
  'ByggaarMax': None,
  'EtageMin': None,
  'EtageMax': None,
  'PlanMin': None,
  'PlanMax': None

In [59]:
#urlreq = 'https://home.dk/resultatliste/?q=2200+K%C3%B8benhavn+N:33'
urlreq = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=15&q=2200%20K%C3%B8benhavn%20N&Energimaerker=null&SearchType=0&_=1571481546474'
response = urllib.request.urlopen(urlreq)
#req = urllib.request.Request(urlreq)
#with urllib.request.urlopen(req) as response:
#   the_page = response.read()
#print(the_page)
# load as json
response.read()#.decode('utf-8')
#jresponse = json.load(response, encoding='utf-8')
#json.loads(line.decode("utf-8","ignore"))
# write to file as pretty print
#with open('asdaresp.json', 'w') as outfile:
#    json.dump(jresponse, outfile, sort_keys=True, indent=4)

b'\xd5\xbd\xddr\x1c\xc7\x96\xa5\xf9*\x18\\\xf4\x95\x1c\x8cp\x0f\xf7\x88\xa0YY\x1bO\x89\xfa)Q?FR*\x1b+k\xa3yDx\x80\x10A\x80\x93\x99\x90\x8eN\xd9\xb9\x9c~\x86\xba\x9c\x9bj\xb3y\x83\xba\x1e\xbd\xd8|;\tGF\xa6\xa7\x8e\xe8$\x11\xec\xd0\xe9f\x91 \xc0\x05\xe4\xca\xf0\x9f\xbd\xd7^\xeb\xdfOWa\xb8X\x85~\xf3\xe3\xea\xf2\xf4\xe1\xd5\xcd\xe5\xe5g\xa7\x17Won6\xdf^\x0f\x81\x8f\xfc\xfb\xe9\xb3\xeb\xd5\xe6\xf9ooB\xfc[\xf9\xf3\xf7\xab!\xac\xe2\x07\xfe\xf9f\xb5\nW\x9b\x1f\xfcy\xf8\xee\xe6u\'\x7fQ|v\xfa,\xf8U\xff\xf2iX\xdf\\n\xd6?\x84\x95\xfc\xf5\xe9\xc3\xd2~v\xfa\x7f\x9d><\xd5\xba(N\xbe\xf9\xfd\xbf\xbap\xf5\xd2\xffru\xf2\xdd\xe9g\xa7\x8f\x7f\x0eW\xc3\xf5\xeb\xf5\x06\xb0\x9f\xca\xf8\xafO?\xfa\xf4\xabc\x1f}\xfc\xe4\xd8G\x7f:\xfa\xd1G\x8f\x8e}\xee\x0fG?\xf7\x8b\xa3hO\xbe?\xf6/|\xf5\xe5\xb1\x8f~q\xf4\xa3\xdf\xdd\xa1}q\xcd+\xb7\xb9\xba\xb8:\xbf\xba{5\x7fX]\xff\x0c\x1d\xdf\xf1\xf2\x7f=\xc4\x7f\xf4\xfb\xab\xcb\xdf\xfe\xb2\xba~\x15V_]\xbf\xbec\xe2\x87\xd5E\x1f\xbe\xbd\xb8\x8a\x9f\xf5\xf6\xcf\xfe\xaf\xf1\xcf\xbct\

In [ ]:
GET /resultatliste/?q=2200+K%C3%B8benhavn+N HTTP/1.1
Host: home.dk
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36
Sec-Fetch-Mode: navigate
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3
Sec-Fetch-Site: same-origin
Referer: https://home.dk/
Accept-Encoding: gzip, deflate, br
Accept-Language: da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: _ga=GA1.2.55867073.1571476775; _gid=GA1.2.440813985.1571476775; _gcl_au=1.1.826145108.1571476775; adv_guid=bb3bdd31-b9117e-fea1b-b23eb8-44b028e|ADV; CookieInformationConsent=%7B%22website_uuid%22%3A%22bfb17c80-64c9-4e36-bca8-739bd5bf03ee%22%2C%22timestamp%22%3A%222019-10-19T09%3A19%3A37.172Z%22%2C%22consent_url%22%3A%22https%3A%2F%2Fhome.dk%2F%22%2C%22consent_website%22%3A%22home.dk%22%2C%22consent_domain%22%3A%22home.dk%22%2C%22user_uid%22%3A%220e7584f9-4838-44f8-996a-90f14c9fc36c%22%2C%22consents_approved%22%3A%5B%22cookie_cat_necessary%22%2C%22cookie_cat_functional%22%2C%22cookie_cat_statistic%22%2C%22cookie_cat_marketing%22%2C%22cookie_cat_unclassified%22%5D%2C%22consents_denied%22%3A%5B%5D%2C%22user_agent%22%3A%22Mozilla%2F5.0%20%28Macintosh%3B%20Intel%20Mac%20OS%20X%2010_14_6%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F77.0.3865.120%20Safari%2F537.36%22%7D; ASP.NET_SessionId=eedeczunm3i0d4vnrcsipwvy


In [28]:

urlreq = 'https://home.dk/umbraco/backoffice/home-api/SEARCH?CurrentPageNumber=0&SearchResultsPerPage=15&q=2200%20K%C3%B8benhavn%20N&Energimaerker=null&SearchType=0&_=1571481546474'
#response = urllib.request.urlopen(urlreq)
req = urllib.request.Request(urlreq)
with urllib.request.urlopen(req) as response:
   the_page = response.read()
print(the_page)

b'\xd5\xbd\xddr\x1c\xc7\x96\xa5\xf9*\x18\\\xf4\x95\x1c\x8cp\x0f\xf7\x88\xa0YY\x1bO\x89\xfa)Q?FR*\x1b+k\xa3yDx\x80\x10A\x80\x93\x99\x90\x8eN\xd9\xb9\x9c~\x86\xba\x9c\x9bj\xb3y\x83\xba\x1e\xbd\xd8|;\tGF\xa6\xa7\x8e\xe8$\x11\xec\xd0\xe9f\x91 \xc0\x05\xe4\xca\xf0\x9f\xbd\xd7^\xeb\xdfOWa\xb8X\x85~\xf3\xe3\xea\xf2\xf4\xe1\xd5\xcd\xe5\xe5g\xa7\x17Won6\xdf^\x0f\x81\x8f\xfc\xfb\xe9\xb3\xeb\xd5\xe6\xf9ooB\xfc[\xf9\xf3\xf7\xab!\xac\xe2\x07\xfe\xf9f\xb5\nW\x9b\x1f\xfcy\xf8\xee\xe6u\'\x7fQ|v\xfa,\xf8U\xff\xf2iX\xdf\\n\xd6?\x84\x95\xfc\xf5\xe9\xc3\xd2~v\xfa\x7f\x9d><\xd5\xba(N\xbe\xf9\xfd\xbf\xbap\xf5\xd2\xffru\xf2\xdd\xe9g\xa7\x8f\x7f\x0eW\xc3\xf5\xeb\xf5\x06\xb0\x9f\xca\xf8\xafO?\xfa\xf4\xabc\x1f}\xfc\xe4\xd8G\x7f:\xfa\xd1G\x8f\x8e}\xee\x0fG?\xf7\x8b\xa3hO\xbe?\xf6/|\xf5\xe5\xb1\x8f~q\xf4\xa3\xdf\xdd\xa1}q\xcd+\xb7\xb9\xba\xb8:\xbf\xba{5\x7fX]\xff\x0c\x1d\xdf\xf1\xf2\x7f=\xc4\x7f\xf4\xfb\xab\xcb\xdf\xfe\xb2\xba~\x15V_]\xbf\xbec\xe2\x87\xd5E\x1f\xbe\xbd\xb8\x8a\x9f\xf5\xf6\xcf\xfe\xaf\xf1\xcf\xbct\